In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from copy import deepcopy
from tqdm import tqdm

from ex_simulation import p
from dset import get_dataloader, load_pretrained_model

# adaptive-wavelets modules
from losses import get_loss_f
from train import Trainer
from evaluate import Validator
from transform1d import DWT1d
from utils import get_1dfilts, get_wavefun
from wave_attributions import Attributer
from visualize import cshow, plot_1dfilts, plot_1dreconstruct

### interpretation modules
sys.path.append('../../lib/trim')
from trim import TrimModel
import acd
from acd.scores import cd_propagate
from acd.scores import cd

# load data and model

In [2]:
# load data and model
train_loader, test_loader = get_dataloader(p.data_path,
                                           batch_size=p.batch_size)

model = load_pretrained_model(p.model_path)

In [4]:
# wavelet transform 
wt = DWT1d(wave=p.wave, mode='zero', J=p.J, init_factor=1, noise_factor=0).to(device)

# get image
data = iter(test_loader).next()[0].to(device)
data_t = wt(data)
recon = wt.inverse(data_t)

print("Reconstruction error={:.5f}".format(torch.norm(recon - data)**2/data.size(0)))

# get 1d wavelet filters
filt = get_1dfilts(wt)

Reconstruction error=0.00000


In [5]:
print('shape of data / recon', data.shape, recon.shape)
for i in range(len(data_t)):
    print('shape of wavelet coeffs', data_t[i].shape)

shape of data / recon torch.Size([100, 1, 64]) torch.Size([100, 1, 64])
shape of wavelet coeffs torch.Size([100, 1, 12])
shape of wavelet coeffs torch.Size([100, 1, 36])
shape of wavelet coeffs torch.Size([100, 1, 22])
shape of wavelet coeffs torch.Size([100, 1, 15])
shape of wavelet coeffs torch.Size([100, 1, 12])


In [6]:
print('reconstruction error: {:.5f}'.format(torch.norm(data - recon).item()/data.size(0)))

reconstruction error: 0.00000


In [7]:
# trim model
mt = TrimModel(model, wt.inverse, use_residuals=True).to(device)
out = mt(data_t, deepcopy(data))
print(out[:5] - model(data)[:5])

tensor([[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]], device='cuda:0', grad_fn=<SubBackward0>)


# calculate cd score in original domain

In [8]:
data = data[:30]
mask = torch.zeros_like(data)
mask[:,0,0] = 1

In [1]:
rel, irrel = cd.cd(data, model, mask, model_type=None, device='cuda')

NameError: name 'cd' is not defined

In [ ]:
cd.cd

# compute cd score in wavelet domain

In [ ]:
data_t = wt(data) # select some coefs of data_t to interpret (so maybe zero all but one coefs)
rel = wt.inverse(data_t)
irrel = data - rel